In [4]:
# Hugging face testing
from transformers import AutoTokenizer, TFAutoModelForMaskedLM
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

model = TFAutoModelForMaskedLM.from_pretrained("bert-base-chinese")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [9]:
def testModel(sentence, k = 5):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='tf')

    # Get the position of the [MASK] token
    mask_token_index = tf.where(inputs['input_ids'] == tokenizer.mask_token_id)[0, 1]

    # Generate model predictions for the masked token
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    predictions = outputs.logits[0, mask_token_index]

    # Convert the predictions to probabilities
    probabilities = tf.nn.softmax(predictions, axis=-1)

    # Get the top-k predicted tokens and their probabilities
    top_k_values, top_k_indices = tf.math.top_k(probabilities, k=k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.numpy())

    # Print the top-k predicted tokens and their probabilities
    print(f"Top-{k} predictions:")
    for i in range(k):
        print(f"{predicted_tokens[i]}: {top_k_values[i].numpy()}")
        
# Define the input sentence with [MASK] token
sentence = "今天的天氣很清[MASK]舒服"
testModel(sentence)

Top-5 predictions:
爽: 0.6484012007713318
新: 0.16201545298099518
涼: 0.08563422411680222
朗: 0.0348283126950264
淨: 0.006423946004360914


In [17]:
# Try fine tune it with own data
from datasets import load_dataset

dataset = load_dataset("csv", data_files="../dataset/set_01_02_new.csv")

print(dataset)


ModuleNotFoundError: No module named 'datasets'

In [15]:
# Set up tokenizer
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"])

dataset = df.transform(tokenize_dataset)
tf_dataset = model.prepare_tf_dataset(dataset, batch_size=16, shuffle=True, tokenizer=tokenizer)


AttributeError: 'DataFrame' object has no attribute 'map'